### This is some preliminary analysis of this dataset now that everything is parsed correctly

In [88]:
import os
import numpy as np
import pandas as pd
%matplotlib qt
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('dark_background')
from scipy.optimize import curve_fit

In [2]:
df = pd.read_csv('df_parsed_p_times.csv')

In [3]:
df.day = np.around(df.day, 2)

In [33]:
df['color'] = np.random.randint(0, 20, len(df))
df = df.loc[df.Intensity_IntegratedIntensity_cebp <= df.Intensity_IntegratedIntensity_cebp.quantile(.95)]
df = df.loc[df.Intensity_IntegratedIntensity_cebp >= df.Intensity_IntegratedIntensity_cebp.quantile(.05)]
f = df.loc[df.Intensity_IntegratedIntensity_ppar <= df.Intensity_IntegratedIntensity_ppar.quantile(.95)]
df = df.loc[df.Intensity_IntegratedIntensity_ppar >= df.Intensity_IntegratedIntensity_ppar.quantile(.05)]

In [57]:
'''first thing that I would like to do is look at the PPARg'''
ax = sns.lineplot(df.day, df.Intensity_IntegratedIntensity_ppar, color = 'magenta')

In [58]:
'''lineplot for cebpb'''
ax = sns.lineplot(df.day, df.Intensity_IntegratedIntensity_cebp, color = 'lawngreen')

In [70]:
'''strip plot of the ppar gamma'''
ax = sns.stripplot(df.day, df.Intensity_IntegratedIntensity_ppar, hue = df.color, palette= sns.color_palette('Purples', 20))
ax = sns.pointplot(df.day, df.Intensity_IntegratedIntensity_ppar, ci = 'sd', color = 'w', errwidth = 5, capsize = .2)
plt.legend([])

In [69]:
'''strip plot of the cebp beta'''
ax = sns.stripplot(df.day, df.Intensity_IntegratedIntensity_cebp, hue = df.color, palette= sns.color_palette('Greens', 20))
plt.legend([])
ax = sns.pointplot(df.day, df.Intensity_IntegratedIntensity_cebp, ci = 'sd', color = 'w', errwidth = 5, capsize=.2)

#ax = sns.boxplot(df.day, df.Intensity_IntegratedIntensity_cebp, fliersize = 0)

In [40]:
ax = sns.stripplot(df.day, df.AreaShape_Area, color = 'm')
ax = sns.pointplot(df.day, df.AreaShape_Area, ci = 'sd')

In [56]:
sns.lineplot(df.day, df.Intensity_IntegratedIntensity_cebp, color = 'lawngreen')
plt.legend(['CEBPb'])
ax2 = plt.twinx()
sns.lineplot(df.day, df.Intensity_IntegratedIntensity_ppar, color = 'magenta', ax = ax2)
plt.legend(['PPARg'])

In [73]:
df6 =  (df.loc[df.day == 6])

In [74]:
sns.rugplot(df6.Intensity_IntegratedIntensity_ppar)

In [80]:
n, bins, patches = plt.hist(df6.Intensity_IntegratedIntensity_ppar, cumulative = -1)
plt.plot(bins[:-1], n)

### I would like to see if I can now fit an exponential decay to the 1-cdf

In [146]:
y_hist, x_hist, patch = plt.hist(df.Intensity_IntegratedIntensity_ppar,  density = 1, cumulative = -1,  bins = 300)
x_hist = x_hist[:-1]


In [147]:
plt.plot(x_hist, y_hist)

In [148]:
'''single component exponential decay'''
def one_comp_exp_decay(t, A, tau):
    return(A * np.exp(-t / tau))

In [150]:
popt1, pcov1 = curve_fit(one_comp_exp_decay, x_hist, y_hist)

In [151]:
'''two component exponential decay'''
def two_comp_exp_decay(t, f_slow, tau_slow, tau_fast):
    return(f_slow * np.exp(-t / tau_slow) + (1-f_slow) * np.exp(-t / tau_fast))

In [152]:
popt2, pcov2 = curve_fit(two_comp_exp_decay, x_hist, y_hist)

In [155]:
#plt.hist(df6.Intensity_IntegratedIntensity_ppar, density=1, cumulative= -1, bins = 300)
plt.plot(x_hist,  one_comp_exp_decay(x_hist, *popt1))
#plt.plot(x_hist, two_comp_exp_decay(x_hist, *popt2), color = 'r')

In [132]:
'''two component exponential decay'''
def three_comp_exp_decay(t, f1, tau1, f2, tau2, tau3):
    return(f1 * np.exp(-t / tau1) + f2 * np.exp(-1 / tau2) + (1 - f1 - f2) * np.exp(-1 / tau3))

In [133]:
popt3, pcov3 = curve_fit(three_comp_exp_decay, x_hist, y_hist)

C:\Users\LegantLab\anaconda3\lib\site-packages\scipy\optimize\minpack.py:808: OptimizeWarning: Covariance of the parameters could not be estimated
  category=OptimizeWarning)


In [134]:
'''two component exponential decay'''
def exp_decay_two(t, A1, tau1, A2, tau2):
    return(A1 * np.exp(-t / tau1) + A2 * np.exp(-t / tau2))

In [121]:
popt2, pcov2 = curve_fit(exp_decay_two, x_hist, y_hist)

In [135]:
plt.plot(x_hist, three_comp_exp_decay(x_hist, *popt3))

In [156]:
sns.kdeplot(df6.Intensity_IntegratedIntensity_ppar)

In [157]:
dfp25 = df.loc[df.day == .25]

In [158]:
len(dfp25)

146

In [161]:
sns.distplot(dfp25.Intensity_IntegratedIntensity_cebp, bins = 20)